**Instituto de Informática - UFG**

Disciplina: Inteligência Artificial

Prof. Cedric Luiz de Carvalho

Grupo número (   )

---


Aluna(o): 

---

Aluna(o): 

---

Aluna(o): 

---

# Especificação do problema : Árvores de Decisão

A partir da base de dados disponibilizada anteriormente, induzir uma árvore de decisão que correlacione o desempenho nos dois primeiros períodos do curso com um bom desempenho final e permita fazer previsões sobre o desempenho final de novos alunos.

Para isso, considere:


1.  Um aluno com bom desempenho é aquele que se forma em até 4 anos e meio ou se forma com média igual ou superior a 7 ou  se forma com número de reprovações igual inferior a 5.

2. Para o treinamento do modelo, considere apenas as disciplinas dos dois primeiros períodos do curso.

3. Escolher pelo menos um dos seguintes algoritmos para a implementação: ID3, C4.5 e CART.

4. Para visualização das árvores de decisão geradas, utilize o [graphviz](https://towardsdatascience.com/interactive-visualization-of-decision-trees-with-jupyter-widgets-ca15dd312084).


**Observação 1**: Poderão ser utilizados códigos de outros autores, desde que sejam feitos ou devidos créditos aos mesmos. Esses códigos devem ser devidamente comentados.

**Observação 2**: O trabalho deve ser feito em grupo, mas a avaliação será feita individualmente. Para isso, o grupo deverá fazer um planejamento/cronograma de trabalho e o trabalho de cada membro deverá ser devidamente identificado (por meio de** ##NomeDoAluno**)

#Procedimento


Cada membro do grupo deve assumir um dos papéis a seguir:



1.   Gerente do projeto: deve se encarregar de organizar as atividades do projeto, produzir toda a documentação do mesmo, seguindo rigorosamente o padrão definido pelo professor e fazer sua apresentação no momento apropriado.
2.   Analista de dados: deve produzir o conjunto de exemplos necessário para a implementação do modelo, bem como a sua documentação.
3. Desenvolvedor: deve implementar o modelo proposto pelo grupo, seja utilizando códigos de terceiros ou seus próprios códigos.


In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import math
df = pd.read_csv('baseDeDados.csv', sep=';')
df.head()

/home/user/.conda/envs/tcc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,ano_nascimento_discente,idade_conclusao_ensino_medio,idade_ingresso_universidade,idade_colacao_grau,uf_naturalidade_discente,discente_estudou_escola_publica?,ano_conclusao_ensino_medio,nome_municipio_curso,nome_campus_curso,...,semestre_previsto_componente,ch_total_componente,frequencia_discente_componente,nota_discente_componente,situacao_discente_componente,quantidade_trancamentos,quantidade_reprovacoes_nc,quantidade_reprovacoes_nl,quantidade_reprovacoes_neobr,quantidade_reprovacoes_neopt
0,1,1989,17.0,19,26.0,GO,NÃO,2006.0,GOIÂNIA,SAMAMBAIA,...,1,64,0.91,7.3,AP,1.0,6.0,9.0,10.0,NaN
1,1,1989,17.0,19,26.0,GO,NÃO,2006.0,GOIÂNIA,SAMAMBAIA,...,1,64,0.91,6.8,AP,1.0,6.0,9.0,10.0,NaN
2,1,1989,17.0,19,26.0,GO,NÃO,2006.0,GOIÂNIA,SAMAMBAIA,...,1,64,1.00,5.0,AP,1.0,6.0,9.0,10.0,NaN
3,1,1989,17.0,19,26.0,GO,NÃO,2006.0,GOIÂNIA,SAMAMBAIA,...,2,64,0.94,5.0,AP,1.0,6.0,9.0,10.0,NaN
4,1,1989,17.0,19,26.0,GO,NÃO,2006.0,GOIÂNIA,SAMAMBAIA,...,2,64,0.91,4.2,RM,1.0,6.0,9.0,10.0,NaN


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22362 entries, 0 to 22361
Data columns (total 66 columns):
id                                            22362 non-null int64
ano_nascimento_discente                       22362 non-null int64
idade_conclusao_ensino_medio                  18632 non-null float64
idade_ingresso_universidade                   22362 non-null int64
idade_colacao_grau                            8223 non-null float64
uf_naturalidade_discente                      22204 non-null object
discente_estudou_escola_publica?              21326 non-null object
ano_conclusao_ensino_medio                    18632 non-null float64
nome_municipio_curso                          22362 non-null object
nome_campus_curso                             22362 non-null object
nome_curso                                    22362 non-null object
grau_academico                                22362 non-null object
modalidade                                    22362 non-null object
turno   

In [6]:
# 0 significa mau aluno
# 1 significa bom aluno
# Todos começam como mau aluno
df['classificacao'] = 0
df[['classificacao']].head()

,classificacao
0,0
1,0
2,0
3,0
4,0


In [86]:
# " Um aluno com bom desempenho é aquele que se forma em até 4 anos e meio"

# Desconsiderar alunos que ingressaram há menos de 4 anos e meio atrás: depois de 2015.1

alunos = df[['id', 'ano_periodo_ingresso', 'data_colacao_grau', 'media_global_atual_discente']].\
rename(columns={'id': 'identificador'})

alunos['reprovacoes'] = 0

cols = ['quantidade_reprovacoes_nc', 'quantidade_reprovacoes_nl', 'quantidade_reprovacoes_neobr', 'quantidade_reprovacoes_neopt']

media_global = df[['id', 'nota_discente_componente']].\
rename(columns={'id': 'identificador'}).\
dropna().\
groupby(by=['identificador']).\
mean().\
rename(columns={'nota_discente_componente': 'media_global_calculada'})

alunos = pd.merge(alunos, media_global, how='outer', on='identificador')

alunos['media_global_considerada'] = alunos['media_global_atual_discente'].\
combine(alunos['media_global_calculada'], lambda x,y: x if not math.isnan(x) else y )

for item in cols:
     alunos['reprovacoes'] += df[item].fillna(0)

alunos['classificacao_1'] = 0
alunos['classificacao_2'] = 0
alunos['classificacao_3'] = 0
alunos['classificacao_final'] = 0

alunos = alunos[alunos['ano_periodo_ingresso'] < 2015.1 ]

alunos['data_colacao_grau'] = alunos['data_colacao_grau'].fillna('0-0-0')

alunos['data_colacao_grau'] = alunos['data_colacao_grau'].\
apply(lambda x: int(x.split('-')[0]) + 0.1 * (int(x.split('-')[1]) < 7) + 0.2 * (int(x.split('-')[1]) > 7)  )

alunos['diff'] = alunos['data_colacao_grau'] - alunos['ano_periodo_ingresso']

alunos['classificacao_1'] = alunos['diff'].\
apply(lambda x: 1 if ( x <= 4.5 and x > 0 ) else 0)

alunos = alunos.drop_duplicates()

# se forma

alunos['formado'] = alunos['diff'].\
apply(lambda x: 1 if x > 0 else 0)

# "se forma E tem média igual ou superior a 7"

alunos['classificacao_2'] = alunos['media_global_considerada'].\
apply(lambda x: 1 if x >= 7 else 0 ).\
combine(alunos['formado'], lambda x,y: 1 if x & y else 0 )

# "se forma E tem número de reprovações igual inferior a 5."

alunos['classificacao_3'] = alunos['reprovacoes'].\
apply(lambda x: 1 if x <= 5 else 0 ).\
combine(alunos['formado'], lambda x,y: 1 if x & y else 0 )

# classificacao final

alunos['classificacao_final'] = alunos['classificacao_1'].\
combine(alunos['classificacao_2'], lambda x,y: 1 if x | y else 0 ).\
combine(alunos['classificacao_3'], lambda x,y: 1 if x | y else 0 )

alunos.sample(10)

,identificador,ano_periodo_ingresso,data_colacao_grau,media_global_atual_discente,reprovacoes,media_global_calculada,media_global_considerada,classificacao_1,classificacao_2,classificacao_3,classificacao_final,diff,formado
3516,90,2009.2,0.1,NaN,6.0,1.457143,1.457143,0,0,0,0,-2009.1,0
12996,371,2012.2,0.1,4.98,30.0,4.850000,4.980000,0,0,0,0,-2012.1,0
2195,57,2009.2,2017.2,5.48,23.0,5.443548,5.480000,0,0,0,0,8.0,1
15398,444,2013.1,0.1,3.84,2.0,3.550000,3.840000,0,0,0,0,-2013.0,0
12281,352,2012.2,2017.2,6.83,9.0,6.878000,6.830000,0,0,0,0,5.0,1
3397,85,2009.1,0.1,5.14,19.0,0.983333,5.140000,0,0,0,0,-2009.0,0
5045,134,2009.1,0.1,NaN,0.0,8.000000,8.000000,0,0,0,0,-2009.0,0
1136,29,2008.1,2011.1,NaN,1.0,7.531034,7.531034,1,1,1,1,3.0,1
3209,80,2009.1,2013.2,NaN,16.0,5.482456,5.482456,1,0,0,1,4.1,1
5819,159,2010.2,2018.2,4.60,46.0,4.527586,4.600000,0,0,0,0,8.0,1


In [88]:
alunos[['identificador', 'classificacao_final']].head()

,identificador,classificacao_final
0,1,0
56,2,1
96,3,1
142,4,0
190,5,1


In [90]:
# "Para o treinamento do modelo, considere apenas as disciplinas dos dois primeiros períodos do curso."

df[['matriz_semestre_previsto_cursar_compomente', 'nome_componente']].drop_duplicates().head()

,matriz_semestre_previsto_cursar_compomente,nome_componente
0,CICOMP-BI-2/1-IME0073,CÁLCULO 1
1,CICOMP-BI-2/1-INF0131,LÓGICA MATEMÁTICA
2,CICOMP-BI-2/1-INF0135,MATEMÁTICA DISCRETA
3,CICOMP-BI-2/2-IFI0105,FÍSICA PARA COMPUTAÇÃO
4,CICOMP-BI-2/2-IME0006,ÁLGEBRA LINEAR


In [98]:
disciplinas = df[['matriz_semestre_previsto_cursar_compomente', 'nome_componente']].\
drop_duplicates().\
rename(columns={'matriz_semestre_previsto_cursar_compomente': 'semestre_previsto'})

disciplinas


disciplinas['num_semestre'] = disciplinas['semestre_previsto'].\
apply(lambda x: int(x.split('/')[1][0]) )

disciplinas = disciplinas[disciplinas['num_semestre'] <= 2]
# somente da grade antiga
disciplinas = disciplinas[disciplinas['semestre_previsto'].str.contains('CICOMP-BI-2')]

disciplinas

,semestre_previsto,nome_componente,num_semestre
0,CICOMP-BI-2/1-IME0073,CÁLCULO 1,1
1,CICOMP-BI-2/1-INF0131,LÓGICA MATEMÁTICA,1
2,CICOMP-BI-2/1-INF0135,MATEMÁTICA DISCRETA,1
3,CICOMP-BI-2/2-IFI0105,FÍSICA PARA COMPUTAÇÃO,2
4,CICOMP-BI-2/2-IME0006,ÁLGEBRA LINEAR,2
6,CICOMP-BI-2/2-IME0078,CÁLCULO 2,2
7,CICOMP-BI-2/2-INF0157,PROGRAMAÇÃO DE COMPUTADORES 2 CN,2
8,CICOMP-BI-2/2-INF0195,SISTEMAS DIGITAIS,2
58,CICOMP-BI-2/1-INF0109,INTRODUÇÃO À COMPUTAÇÃO,1
60,CICOMP-BI-2/1-INF0156,PROGRAMAÇÃO DE COMPUTADORES 1 CN,1


In [126]:
disciplinas = disciplinas[['nome_componente']]

identificadores = alunos[['identificador']]
notas = pd.merge(df[['id', 'nome_componente', 'nota_discente_componente']].\
                       rename(columns={'id': 'identificador'}),\
                       identificadores, how='outer', on='identificador')
# enumerar linhas repetidas
notas['componente_enum'] = notas[['identificador', 'nome_componente']].groupby(by=['identificador', 'nome_componente']).cumcount()+1
notas.head(15)

,identificador,nome_componente,nota_discente_componente,componente_enum
0,1,CÁLCULO 1,7.3,1
1,1,LÓGICA MATEMÁTICA,6.8,1
2,1,MATEMÁTICA DISCRETA,5.0,1
3,1,FÍSICA PARA COMPUTAÇÃO,5.0,1
4,1,ÁLGEBRA LINEAR,4.2,1
5,1,ÁLGEBRA LINEAR,6.9,2
6,1,CÁLCULO 2,5.2,1
7,1,PROGRAMAÇÃO DE COMPUTADORES 2 CN,6.0,1
8,1,SISTEMAS DIGITAIS,6.3,1
9,1,ARQUITETURA DE COMPUTADORES,5.8,1


In [166]:
# Vamos descobrir:
# -- qual a média dos bons alunos que estão fazendo pela primeira vez
# -- qual a média dos bons alunos que estão fazendo pela segunda vez
# -- qual a média dos bons alunos que estão fazendo pela terceira vez
# etc. etc.

identificadores = alunos[['identificador', 'classificacao_final']]
identificadores = identificadores[identificadores['classificacao_final'] == 1][['identificador']]


medias = notas[['identificador','nome_componente','nota_discente_componente','componente_enum']]

medias = \
pd.merge(identificadores, notas, how='outer', on='identificador').\
groupby(by=['nome_componente', 'componente_enum']).\
mean().\
reset_index([0,-1]).\
rename(columns={'nota_discente_componente' : 'media'})

medias 
# Qual porcentagem de bons alunos fizeram essa matéria pela n-ésima vez?

contagem = pd.merge(identificadores, notas, how='outer', on='identificador').\
groupby(by=['nome_componente', 'componente_enum']).\
count().reset_index([0,-1])

porcentagem = contagem.\
groupby('nome_componente').transform(lambda x: x / x.sum() )[['nota_discente_componente']].\
rename(columns={'nota_discente_componente' : 'porcentagem'})

bons_alunos = medias[['nome_componente', 'componente_enum']]
bons_alunos['porcentagem'] = porcentagem['porcentagem']
bons_alunos['media'] = medias['media']
bons_alunos
#groupby(by=['nome_componente', 'componente_enum']).\
#transform(lambda x: x / x.sum() )

/home/user/.conda/envs/tcc/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,nome_componente,componente_enum,porcentagem,media
0,ALGORITMOS E ESTRUTURAS DE DADOS 1,1,0.921569,6.612766
1,ALGORITMOS E ESTRUTURAS DE DADOS 1,2,0.078431,5.950000
2,ALGORITMOS E ESTRUTURAS DE DADOS 2,1,1.000000,5.973333
3,ANÁLISE E PROJETO DE ALGORITMOS,1,0.620042,4.424242
4,ANÁLISE E PROJETO DE ALGORITMOS,2,0.227557,3.770642
5,ANÁLISE E PROJETO DE ALGORITMOS,3,0.110647,4.801887
6,ANÁLISE E PROJETO DE ALGORITMOS,4,0.031315,4.746667
7,ANÁLISE E PROJETO DE ALGORITMOS,5,0.008351,4.950000
8,ANÁLISE E PROJETO DE ALGORITMOS,6,0.002088,6.000000
9,ARQUITETURA DE COMPUTADORES,1,0.780570,5.762876


In [168]:
# Para cada par (matéria, aluno) devemos:
# -- Atribuir um score que é a porcentagem de bons alunos na última repetição da matéria
# -- Atribuir o desvio da nota do aluno em relação à média dos bons alunos naquela repetição
# -- Atribuir 0 ( se for mau aluno) ou 1 ( se for bom aluno )
# ( repetições sem bons alunos são desconsideradas )
# esses atributos serão fornecido

notas.head()

,identificador,nome_componente,nota_discente_componente,componente_enum
0,1,CÁLCULO 1,7.3,1
1,1,LÓGICA MATEMÁTICA,6.8,1
2,1,MATEMÁTICA DISCRETA,5.0,1
3,1,FÍSICA PARA COMPUTAÇÃO,5.0,1
4,1,ÁLGEBRA LINEAR,4.2,1
